In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import warnings
warnings.filterwarnings(action='ignore')
import koreanize_matplotlib

# 환율정보 DB에서 추출하기

In [2]:
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb

<function pymysql.install_as_MySQLdb()>

In [3]:
!pip install cryptography

In [4]:
# MySQL 연결 설정 ver 1
connection = pymysql.connect(
    host='localhost',    # MySQL 서버 주소 (로컬 호스트)
    user='root',         # MySQL 사용자 이름 (root)
    password='1234',  # MySQL 비밀번호를 입력하세요
    database='korea_exchange_rate',  # 사용할 데이터베이스 이름
    port=3306,           # MySQL 기본 포트 번호
    charset='utf8mb4'
)

# 쿼리 실행 및 데이터 가져오기
query = "SELECT * FROM exchange_rate"  # 테이블의 모든 데이터를 가져오는 쿼리
exchange_rate= pd.read_sql(query, connection)

# MySQL 연결 종료
connection.close()

# 데이터 출력
print(exchange_rate.head())


         date            통화  현찰_살때_환율  현찰_살때_Spread  현찰_팔때_환율  현찰_팔때_Spread  \
0  1995-01-03        미국 USD    800.53           1.5    776.87           1.5   
1  1995-01-03  일본 JPY (100)    798.89           1.5    775.29           1.5   
2  1995-01-03        유로 EUR      0.00           0.0      0.00           0.0   
3  1995-01-03        중국 CNY      0.00           0.0      0.00           0.0   
4  1995-01-03        홍콩 HKD    103.45           3.0    100.41           3.0   

   송금_보낼때  송금_받을때  T/C_살때  외화_수표_팔때   매매기준율  환가_료율  미화 환산율  
0  791.80  785.60  792.64      0.00  788.70   0.00  1.0000  
1  791.81  782.37  792.59      0.00  787.09   0.00  0.9980  
2  975.40  959.92    0.00    957.94  967.66   7.37  1.2269  
3   94.12    0.00    0.00      0.00   93.38   1.00  0.1184  
4  102.74  101.12    0.00      0.00  101.93   0.00  0.1292  


In [5]:
exchange_rate.head()

,date,통화,현찰_살때_환율,현찰_살때_Spread,현찰_팔때_환율,현찰_팔때_Spread,송금_보낼때,송금_받을때,T/C_살때,외화_수표_팔때,매매기준율,환가_료율,미화 환산율
0,1995-01-03,미국 USD,800.53,1.5,776.87,1.5,791.80,785.60,792.64,0.00,788.70,0.00,1.0000
1,1995-01-03,일본 JPY (100),798.89,1.5,775.29,1.5,791.81,782.37,792.59,0.00,787.09,0.00,0.9980
2,1995-01-03,유로 EUR,0.00,0.0,0.00,0.0,975.40,959.92,0.00,957.94,967.66,7.37,1.2269
3,1995-01-03,중국 CNY,0.00,0.0,0.00,0.0,94.12,0.00,0.00,0.00,93.38,1.00,0.1184
4,1995-01-03,홍콩 HKD,103.45,3.0,100.41,3.0,102.74,101.12,0.00,0.00,101.93,0.00,0.1292


In [6]:
exchange_rate = exchange_rate.loc[(exchange_rate['date'] >= '1999-01-01') & (exchange_rate['통화']=='미국 USD'), ['date', '통화', '현찰_살때_환율']]
exchange_rate

# 1999-01-01 부터
# 미국 통화만
# 현찰 살때 환율만

,date,통화,현찰_살때_환율
40128,1999-01-01,미국 USD,1233.49
40150,1999-01-04,미국 USD,1215.56
40172,1999-01-05,미국 USD,1193.54
40194,1999-01-06,미국 USD,1185.85
40216,1999-01-07,미국 USD,1178.17
...,...,...,...
383346,2024-09-19,미국 USD,1351.74
383404,2024-09-20,미국 USD,1357.34
383462,2024-09-23,미국 USD,1362.53
383520,2024-09-24,미국 USD,1359.38


In [7]:
exchange_rate = exchange_rate.reset_index(drop=True)
exchange_rate

# 인덱스를 0부터 다시 할당하는 과정 : reset_index

,date,통화,현찰_살때_환율
0,1999-01-01,미국 USD,1233.49
1,1999-01-04,미국 USD,1215.56
2,1999-01-05,미국 USD,1193.54
3,1999-01-06,미국 USD,1185.85
4,1999-01-07,미국 USD,1178.17
...,...,...,...
8019,2024-09-19,미국 USD,1351.74
8020,2024-09-20,미국 USD,1357.34
8021,2024-09-23,미국 USD,1362.53
8022,2024-09-24,미국 USD,1359.38


# 금리 데이터 불러오기

In [8]:
# CSV 파일을 불러오기
file_path = './data/기준금리1999-2023.csv'
interest_rate = pd.read_csv(file_path, encoding='utf-8')  # 또는 'cp949'
interest_rate.head()


,변경일자,Unnamed: 1,기준금리
0,2023,01월 13일,3.50
1,2022,11월 24일,3.25
2,2022,10월 12일,3.00
3,2022,08월 25일,2.50
4,2022,07월 13일,2.25


In [9]:
#interest_rate.columns = ['연도', '월일', '기준금리']
#interest_rate.head()

In [10]:
# 1열(연도)과 2열(월, 일)을 합쳐서 새로운 '변경일자' 열을 생성
interest_rate['변경일자'] = interest_rate.iloc[:, 0].astype(str) + ' ' + interest_rate.iloc[:, 1].astype(str)

# '변경일자'를 datetime 형식으로 변환
interest_rate['변경일자'] = pd.to_datetime(interest_rate['변경일자'], format='%Y %m월 %d일', errors='coerce')

# 2열(월/일) 삭제
interest_rate = interest_rate.drop(interest_rate.columns[1], axis=1)

# 최종 데이터프레임 확인
print(interest_rate.head())

        변경일자  기준금리
0 2023-01-13  3.50
1 2022-11-24  3.25
2 2022-10-12  3.00
3 2022-08-25  2.50
4 2022-07-13  2.25


# 전국 주소 정보 - 우체국 전국 우편번호 데이터

In [11]:
# glob은 경로상에 있는 모든 파일 경로를 리스트로 만들어 줌
from glob import glob

In [16]:
address_path = glob("./data/*.txt")

In [17]:
address_path

[]

In [18]:
pd.read_csv("./data/서울특별시.txt", sep="|")

FileNotFoundError: [Errno 2] No such file or directory: './data/서울특별시.txt'

In [ ]:
address_df = pd.DataFrame()
for area in address_path:
    temp = pd.read_csv(area, sep="|")
    address_df = pd.concat([address_df, temp])
address_df.head()

In [ ]:
address_df.info()

In [ ]:
address_df.info(show_counts=True)

In [ ]:
address_df.columns

In [ ]:
address_df = address_df[['우편번호', '시도', '시군구', '법정동명', '읍면']]
address_df.head()

# 우편번호 중복 제거하기 drop_duplicates(subset=['기준컬럼'])

In [ ]:
address_df = address_df.drop_duplicates(subset=['우편번호'])
address_df.head()

In [ ]:
address_df.reset_index(drop=True)

In [ ]:
address_df.info()

In [ ]:
address_df = address_df.fillna(" ")
address_df.info()

In [ ]:
address_df.head()

# 구우편번호 데이터 불러와 합치기

In [ ]:
old_address_df = pd.read_excel('./data/zipcode_DB/20150707_우편번호(기존형태).xls', header=[0,1])
old_address_df.head()

In [ ]:
old_address_df.columns

In [ ]:
# 멀티인덱스
old_address_df = old_address_df[[(   '우편번호',  'Unnamed: 0_level_1'),
            (   '일련번호',  'Unnamed: 1_level_1'),
            ( '주    소',                  '시도'),
            ( '주    소',                 '시군구'),
            ( '주    소',                 '읍면동'),
            ( '주    소',                   '리')]]

In [ ]:
old_address_df.columns = ['우편번호', '일련번호', '시도', '시군구', '읍면동', '리']
old_address_df

In [ ]:
old_address_df = old_address_df.drop(['일련번호'], axis=1)

In [ ]:
old_address_df = old_address_df.drop_duplicates(subset=['우편번호'])
old_address_df.head()

In [ ]:
old_address_df = old_address_df.fillna(" ")
old_address_df.info()

In [ ]:
address_df

In [ ]:
address_df['읍면동']= address_df['법정동명'] + address_df['읍면']
address_df

In [ ]:
address_df = address_df.drop(['법정동명', '읍면'], axis=1)
address_df

In [ ]:
address_df.columns

In [ ]:
# 컬럼명 변경
address_df = address_df[['우편번호', '시도', '시군구', '읍면동', '리명']]

# 결과 확인
address_df.head()

In [ ]:
address_df = address_df.rename(columns={"리명" : "리"})
address_df.head()

In [ ]:
old_address_df

# 전국 학원 정보 불러오기

In [ ]:
academy= pd.read_csv('./data/exchange/학원교습소정보_2024년08월31일기준.csv',encoding='cp949')
academy

In [ ]:
academy.info()

In [ ]:
old_address_df[old_address_df['우편번호'] == 135998]

In [ ]:
address_df[address_df['우편번호'] == 6207]

In [ ]:
for i in list(academy['시설도로명상세주소'].unique())[:100]:
    print(i)

In [ ]:
import re
import pandas as pd
import numpy as np

# 예시 데이터 리스트
ex_addresses = academy['시설도로명상세주소'].unique()

# 동, 읍, 면, 리를 추출하는 정규표현식 (숫자가 포함되지 않은 경우만 매칭)
pattern = r'\b([^\d\s]+(동|읍|면|리))\b'

# 추출하기
for address in ex_addresses:
    # NaN 체크 및 문자열 변환
    if isinstance(address, str):
        matches = re.findall(pattern, address)
        for match in matches:
            print(match[0])


### 별로 안좋은걸 알 수 있음 -> 우편번호를 사용하기

In [ ]:
academy['시설도로명우편번호'] = academy['시설도로명우편번호'].apply(lambda x: str(x).strip())

# 우편번호 패턴 파악

In [ ]:
# 신우편번호 범위
print(address_df['우편번호'].min(), address_df['우편번호'].max())

In [ ]:
# 구우편번호 범위
print(old_address_df['우편번호'].min(), old_address_df['우편번호'].max())

In [ ]:
new_post_code_academies = academy[(academy['시설도로명우편번호'].str.len() >= 4) & (academy['시설도로명우편번호'].str.len() <= 5)]
new_post_code_academies

In [ ]:
old_post_code_academies = academy[(academy['시설도로명우편번호'].str.len() == 6)]
old_post_code_academies.head()

# 학원데이터 우편번호로 우편번호 데이터에서 주소 추출

In [ ]:
new_post_code_academies.columns

In [ ]:
address_df['우편번호']

In [ ]:
new_post_code_academies['시설도로명우편번호']

In [ ]:
new_post_code_academies['시설도로명우편번호'].dtype

In [ ]:
new_post_code_academies['시설도로명우편번호'] = new_post_code_academies['시설도로명우편번호'].astype(int)
new_post_code_academies['시설도로명우편번호'].dtype

In [ ]:
new_post_code_academies = pd.merge(new_post_code_academies, address_df, how='left', 
         left_on='시설도로명우편번호', right_on='우편번호')
new_post_code_academies.head()

In [ ]:
old_post_code_academies['시설도로명우편번호'] = old_post_code_academies['시설도로명우편번호'].astype(int)
old_post_code_academies['시설도로명우편번호'].dtype

In [ ]:
old_post_code_academies['시설도로명우편번호'] = old_post_code_academies['시설도로명우편번호'].astype(int)
old_post_code_academies['시설도로명우편번호'].dtype

In [ ]:
old_post_code_academies = pd.merge(old_post_code_academies, old_address_df, how='left', 
         left_on='시설도로명우편번호', right_on='우편번호')
old_post_code_academies.head()

In [ ]:
merged_academy = pd.concat([new_post_code_academies, old_post_code_academies])
merged_academy

In [ ]:
merged_academy['시도'].unique()

In [ ]:
merged_academy['읍면동'].nunique()

In [ ]:
merged_academy = merged_academy.drop(merged_academy[merged_academy['우편번호'].isna() == True].index)
merged_academy

In [ ]:
merged_academy.info()

# 시군구 읍면동 별 학원 수

In [ ]:
academy_num = merged_academy.groupby(['시도', '시군구', '읍면동', '리'])[['학원명']].count().sort_values(by='학원명', ascending=False)
academy_num

In [ ]:
academy_num = academy_num.reset_index()
academy_num

In [ ]:
academy_num = academy_num.rename(columns = {'학원명' : "학원수"})
academy_num.head(50)

# 전국 부동산 실거래가 데이터

In [19]:
apart_file_path = glob("./data/apart1999_202408/*.csv")

In [ ]:
apart_data = pd.DataFrame()
for path in apart_file_path:
    try:
        temp = pd.read_csv(path, header=15, encoding='cp949')
        apart_data = pd.concat([apart_data, temp])
    except Exception as e:
        print(e)
        temp = pd.read_csv(path, header=15, encoding='utf-8')
        apart_data = pd.concat([apart_data, temp])
        
apart_data

In [ ]:
apart_data.to_csv("./apartment_price_1999_2024.csv")

In [ ]:
apart_data = pd.read_csv("./apartment_price_1999_2024.csv")

In [ ]:
apart_data.columns

In [ ]:
(apart_data.isna().sum()/len(apart_data)*100).sort_values(ascending=False)

In [ ]:
apart_data.columns

In [ ]:
apart_data = apart_data.drop(['등기일자', '매도자', '매수자', '동', 'Unnamed: 0', '해제사유발생일', '등기신청일자'], axis=1)
apart_data.head()

In [ ]:
apart_data[apart_data['거래유형'] == '-']

In [ ]:
apart_data = apart_data.drop(['거래유형','중개사소재지', '번지', '부번', '본번', '도로명'], axis=1)

In [ ]:
apart_data = apart_data.reset_index(drop=True)
apart_data.head()

In [ ]:
apart_data = apart_data.drop(['NO'], axis=1)
apart_data

# 아파트 데이터와 학원 데이터 합치기

In [ ]:
academy_num

In [ ]:
apart_data['시군구'].unique()

In [ ]:
def sigungu(x):
    x = x.split()
    if len(x) == 4:
        

In [ ]:
apart_data[['시도', '시군구', '읍면동', '리']] = apart_data['시군구'].str.split(n=3, expand=True)

In [ ]:
academy_num['시도'] = academy_num['시도'].str.replace('전라북도', '전북특별자치도')

In [ ]:
academy_num['시도'] = academy_num['시도'].str.replace('강원도', '강원특별자치도')

In [ ]:
missing_gu = []
for i in apart_data['시군구'].unique():
    if i not in academy_num['시군구'].unique():
        missing_gu.append(i)

print(missing_gu)

In [ ]:
apart_data[[apart_data['시군구'].apply(lambda x : x in missing_gu)]]